In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import skimage as ski
import scipy.sparse
import time

In [ ]:
def forward_differences(M,N):
    row = np.arange(0,M*N)
    dat = np.ones(M*N)
    col = np.arange(0,M*N).reshape(M,N)
    col_xp = np.hstack([col[:,1:], col[:,-1:]])
    col_yp = np.vstack([col[1:,:], col[-1:,:]])
    
    FD1 = scipy.sparse.coo_matrix((dat, (row, col_xp.flatten())), shape=(M*N, M*N))- \
          scipy.sparse.coo_matrix((dat, (row, col.flatten())), shape=(M*N, M*N))

    FD2 = scipy.sparse.coo_matrix((dat, (row, col_yp.flatten())), shape=(M*N, M*N))- \
          scipy.sparse.coo_matrix((dat, (row, col.flatten())), shape=(M*N, M*N))
    
    FD = scipy.sparse.vstack([FD1, FD2])
    
    return FD

In [ ]:
def tgv2_op(M,N):
    FD = forward_differences(M,N)
    FD2 = scipy.sparse.bmat([[FD, None], [None, FD]])
    I = scipy.sparse.eye(2*M*N)
    TGV2 = scipy.sparse.bmat([[FD, -I],[None, FD2]])
    return TGV2

In [ ]:
def prox_l22(u, f, tau):
    return (u + tau*f)/(1+tau)

def proj_inf_l2(p, tau):
    # size must be (K,N), l2 over K, inf over N
    norm_p = np.sqrt(np.sum(p**2, axis=0, keepdims=True))
    p /= np.maximum(1, norm_p/tau)
    return p

def proj_inf(p, tau):
    return np.clip(p, -tau, tau)

In [ ]:
# accelerated primal dual for ROF
def rof_apd(d, lamb=1.0, maxit=1000, check=100, verbose=0):
    
    M,N = d.shape
    d = d.reshape(M*N)
    u = d.copy()

    # dual variable
    p = np.zeros(M*N*2)

    # make nabla operator
    D = forward_differences(M,N)

    # primal and dual step size
    # tau * sigma * L^2 = 1
    L = np.sqrt(8)
    tau = 1/L    
    sigma = 1/tau/L**2
    theta = 1.0
    
    t0 = time.time()
    G = []
    for it in range(0,maxit):

        # remeber old
        u_ = u.copy()

        # primal update
        u -= tau*(D.T@p)
    
        # proximal maps
        u = prox_l22(u,d,tau)
            
        # overrelaxation
        u_ = u + theta*(u-u_)

        theta = 1/np.sqrt(1+tau)
        sigma = sigma/theta;
        tau = tau*theta;
        
        # dual update
        p += sigma*(D@u_)

        # projection
        p = p.reshape(2,M*N)
        p = proj_inf_l2(p, lamb)
        p = p.reshape(2*M*N)
        
        TV1 = lamb*np.sum(np.sqrt(np.sum(((D@u).reshape(2,M*N))**2, axis=0)))
        TV2 = np.sum((D@u)*p)
        gap = TV1 - TV2 + 0.5*np.sum((d-D.T@p-u)**2)
        
        G.append(gap)
        
        if verbose > 0:
            if it%check == check-1:
                

                print("iter = ", it,
                      ", tau = ", "{:.3f}".format(tau),
                      ", sigma = ", "{:.3f}".format(sigma),
                      ", time = ", "{:.3f}".format(time.time()-t0),
                      ", gap = ", "{:.6f}".format(gap),
                      end="\r")
                
    return u.reshape(M,N), p.reshape(2,M,N), np.array(G)

In [ ]:
# Solve TGV2-L2 using primal-dual
def tgv2_pd(d, lamb = (1.0, 2.0), maxit=1000, check=100, verbose=0):
    
    M,N = d.shape
    d = d.reshape(M*N)
  
    # primal variables (u,v)
    u = np.zeros(M*N*3)

    # dual variable
    p = np.zeros(M*N*6)

    # make tgv2 operator
    TGV2 = tgv2_op(M,N)

    # Lipschitz constant of K
    L = np.sqrt(12)

    # primal and dual step size
    # tau * sigma * L^2 = 1
    tau = 1/L*0.1    
    sigma = 1/tau/L**2
    theta = 1.0

    t0 = time.time()
    E=[]
    for it in range(0,maxit):

        # remeber old
        u_ = u.copy()
        
        # primal update
        u = u - tau*(TGV2.T@p)
 
        # proximal maps
        u[:M*N] = prox_l22(u[:M*N],d,tau)
        
        # overrelaxation
        u_ = u + theta*(u-u_)

        # dual update
        p = p + sigma*(TGV2@u_)

        # projections
        p = p.reshape(6, M*N)
        p[:2,:] = proj_inf_l2(p[:2,:], lamb[0])
        p[2:,:] = proj_inf_l2(p[2:,:], lamb[1])
        p = p.reshape(6*M*N)
        
        TGV2_u = (TGV2@u).reshape(6,M*N)
        T1 = np.sum(np.sqrt(np.sum((TGV2_u[:2,:])**2, axis=0)))
        T2 = np.sum(np.sqrt(np.sum((TGV2_u[2:,:])**2, axis=0)))
        
        energy = lamb[0]*T1 + lamb[1]*T2 + 0.5*np.sum((u[:M*N]-d)**2)
        E.append(energy)
        
        if verbose > 0:
            if it%check == check-1:
             
                print("iter = ", it,
                      ", tau = ", "{:.3f}".format(tau),
                      ", sigma = ", "{:.3f}".format(sigma),
                      ", time = ", "{:.3f}".format(time.time()-t0),
                      ", E = ", "{:.6f}".format(energy),
                      end="\r")
                
    return u.reshape(3,M,N), p.reshape(6,M,N), np.array(E)


In [ ]:
# Load image
#g = ski.io.imread("Charlie_Chaplin_portrait.jpg")/255.0
g = ski.io.imread("affine.png")/255.0
M,N = g.shape

# Add noise to the image
f = g + np.random.randn(M,N)*0.1

In [ ]:
# Solve the ROF model using accelerated primal-dual
lamb_tv = 0.1
u_tv, p_tv, _ = rof_apd(f, maxit=1000,lamb=lamb_tv, verbose=1)

In [ ]:
# Solve the TGV2 model using accelerated primal-dual
lamb_tgv2 = 0.1
u_tgv2, p_tgv2, _ = tgv2_pd(f, lamb=(lamb_tgv2, lamb_tgv2*2.0), maxit=1000, verbose=1)

In [ ]:
plt.figure(1, figsize=(15,5))

plt.subplot(131)
plt.imshow(f, cmap="gray")

plt.subplot(132)
plt.imshow(u_tv, cmap="gray")

plt.subplot(133)
plt.imshow(u_tgv2[0], cmap="gray")

ski.io.imsave("denoised_tv.png", np.uint8(np.clip(u_tv*255,0,255)))
ski.io.imsave("denoised_tgv.png", np.uint8(np.clip(u_tgv2[0]*255,0,255)))